In [2]:
import pandas as pd
from dotenv import load_dotenv
import os
from openai import OpenAI

# load_dotenv()
# proj_key = os.getenv('OPENAI_PROJ_KEY')

client = OpenAI()

data = pd.read_csv('~/Documents/SOCY3001/SOCY3001/Literature Review/wosdata.csv')
data = data
data

output = pd.DataFrame(columns=['Location', 'Data', 'Goal', 'Coordinates'])
output = []

In [3]:
for i in range(len(data)):
    prompt = f"Given the following research description, provide the following information:\n\
    1) The location where the research was conducted, if specified. If it is completely impossible to say where the research was conducted, respond with 'Not specified'. Be as specific as possible.\n\
    2) The type of data used in the research (e.g. satellite imagery, remote sensing data). If it is completely impossible to even make a guess, respond with 'Not specified'\n\
    3) The goal of the research (e.g. predicting forest fires, estimating biomass or canopy cover, monitoring biodiversity and habitat quality, classifying tree species). This should specifically be the GOAL or OBJECTIVE of the research, not a general keyword or description of the research conducted.\n\
    4) Your estimation of the coordinates where the research was conducted, separated by ', '. Only provide one set of coordinates. If your answer to (1) was that the location is not specified, respond with 'Not specified' here as well. If the research is being conducted globally or not in a specific place, respond with 'Not specified'. If only a country/general region is specified, make your best guess at the approximate coordinates, or pick a random set of coordinates within that region.\n\
    The coordinates must be formatted like 12.345678, 12.345678. Do not use brackets, letters, extra spaces or punctuation, or anything else. Just the coordinates separated by ', '. You must format the coordinates perfectly - this is absolutely critical, as otherwise everything will break. DO NOT FORMAT THE COORDINATES INCORRECTLY. \n\
    If the research is not related in any way to using artificial intelligence/machine learning/deep learning to study forests/trees/woodlands, return N/A for all four questions. \n\\n\
    Give your response as exactly FOUR values - your answers to each of the four questions - separated by semicolons. Each output MUST have exactly four responses separated by exactly three semicolons - if you don't use exactly 3 semicolons, everything will break, so getting the formatting correct is of critical importance. An example of the output structure is 'China;Satellite images;Forest fire detection;41.40338, 2.17403' (do not use quotes).\n\
    Where possible, the data used and the goal of the research should be general and capable of being aggregated with other, similar research (e.g. 'Remote sensing data' or 'Tree species classification').\n\n\
    Research description:\n{data.iloc[i,0]}\n\
    {data.iloc[i,1]}\n\n\
    {data.iloc[i,2]}"
    response = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages=[{
            "role": "user",
            "content": [
                {"type": "text", "text": prompt}
            ],
        }
        ]
    )
#     col1, col2, col3, col4 = response.choices[0].message.content.split(';')
#     new_row = pd.DataFrame({'Location': [col1], 'Data': [col2], 'Goal': [col3], 'Coordinates': [col4]})
#     output = pd.concat([output, new_row], ignore_index=True)
    output.append(response.choices[0].message.content.split(';')) 

In [9]:
for inner_list in output:
    if inner_list == ['N/A']:
        inner_list[:] = ["N/A", "N/A", "N/A", "N/A"]

for inner_list in output:
    if inner_list == ['New Zealand', 'National level databases', 'Modeling tree diameter', '-40.900557, 174.886', '']:
        inner_list[:] = ['New Zealand', 'National level databases', 'Modeling tree diameter', '-40.900557, 174.886']
    if inner_list == ['India', 'Not specified', 'Landslide susceptibility modeling', '30.vision.at', ' 78.96288']:
        inner_list[:] = ['India', 'Not specified', 'Landslide susceptibility modeling', '30, 78.96288']
    if inner_list == ['Bangladesh', 'Not specified', 'Evaluating regeneration effects of canopy gaps', '23.685', '90.3563']:
        inner_list[:] = ['Bangladesh', 'Not specified', 'Evaluating regeneration effects of canopy gaps', '23.685, 90.3563']
    if inner_list == ['Bangladesh', 'Soil carbon and nutrient data', 'Evaluating soil carbon and nutrient stocks', '23.685', '90.3563']:
        inner_list[:] = ['Bangladesh', 'Soil carbon and nutrient data', 'Evaluating soil carbon and nutrient stocks', '23.685, 90.3563']
    if inner_list == ["Hawai'i", 'Soil carbon data', 'Estimating carbon stocks', '21. resort in a resilient agrofarm model  -2.5 Bank of America calcium', '']:
        inner_list[:] = ["Hawai'i", 'Soil carbon data', 'Estimating carbon stocks', '19.8987, 155.6659']
        

for inner_list in output:
    if len(inner_list) != 4:
        print(inner_list)       

In [15]:
output_clean = output
output_df = pd.DataFrame(output_clean, columns=['Location', 'Data', 'Goal', 'Coordinates'])
names = pd.read_csv('~/Documents/SOCY3001/SOCY3001/Literature Review/wosdata.csv').iloc[:,0]
output_df['Title'] = names

# file_path = '~/Documents/SOCY3001/SOCY3001/Literature Review/output.csv'
# output_df.to_csv(file_path, index=False)

In [11]:
output2 = []

for i in range(len(output_df)):
    prompt = f"For the following study, respond with the category it best fits into from the following options:\n\
    Tree species classification \n\
    Forest cover assessment \n\
    Canopy and biomass estimation \n\
    Habitat degradation and biodiversity monitoring \n\
    Forest fire monitoring and prevention \n\
    Other \n\
    N/A \n\
    Respond with the category ONLY, exactly as written in the prompt. DO NOT INCLUDE NUMBERS, other commentary, or anything else. Do not respond with 'Other' unless the study is not AT ALL related to ANY of the other categories.\n\n\
    Name of study: {output_df.iloc[i,4]}\n\
    Goal of study: {output_df.iloc[i,2]}"
    response = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages=[{
            "role": "user",
            "content": [
                {"type": "text", "text": prompt}
            ],
        }
        ]
    )
#     col1, col2, col3, col4 = response.choices[0].message.content.split(';')
#     new_row = pd.DataFrame({'Location': [col1], 'Data': [col2], 'Goal': [col3], 'Coordinates': [col4]})
#     output = pd.concat([output, new_row], ignore_index=True)
    output2.append(response.choices[0].message.content) 

In [27]:
output3 = []

for i in range(len(output_df)):
    prompt = f"For the following study, respond with the category of data that was used in the study from the following options:\n\
    Drone imagery \n\
    Satellite imagery and LiDAR \n\
    Sensor data \n\
    Ground-level photographs \n\
    Experimental data \n\
    Animal and plant data \n\
    Other \n\
    N/A \n\
    Respond with the category ONLY, exactly as written in the prompt. DO NOT INCLUDE NUMBERS, other commentary, or anything else. Try to make your best guess. Do not respond with 'Other' unless the data does not fit AT ALL into ANY of the other categories. If the input is 'Not specified', respond with 'N/A'\n\n\
    Name of study: {output_df.iloc[i,4]}\n\
    Data used in study: {output_df.iloc[i,1]}"
    response = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages=[{
            "role": "user",
            "content": [
                {"type": "text", "text": prompt}
            ],
        }
        ]
    )
#     col1, col2, col3, col4 = response.choices[0].message.content.split(';')
#     new_row = pd.DataFrame({'Location': [col1], 'Data': [col2], 'Goal': [col3], 'Coordinates': [col4]})
#     output = pd.concat([output, new_row], ignore_index=True)
    output3.append(response.choices[0].message.content) 

In [28]:
output_df['Category'] = output2
output_df['Data category'] = output3
output_df_to_save = output_df[(output_df != "N/A").all(axis=1)]
output_df_to_save.to_csv('~/Documents/SOCY3001/SOCY3001/output.csv', index=False)
output_df_to_save

pd.read_csv('~/Documents/SOCY3001/SOCY3001/output.csv')

,Location,Data,Goal,Coordinates,Title,Category,Data category
0,Not specified,Unmanned aerial vehicle imagery,Improving urban tree species classification,Not specified,Improving Artificial-Intelligence-Based Indivi...,Tree species classification,Drone imagery
1,Konya,Tree height and diameter data,Predicting relationships between individual tr...,"37.87159, 32.48454",Innovative deep learning artificial intelligen...,Tree species classification,Animal or plant data
2,Sweden,Airborne Laser Scanning data,Estimating stem volume and above-ground biomass,coordinates not specified,FOREST BIOPHYSICAL PARAMETER ESTIMATION VIA MA...,Canopy and biomass estimation,LiDAR
3,Turkey,Measured tree height and diameter data,Predicting relationships between total tree he...,"40.1826, 29.0668",Artificial intelligence with deep learning alg...,Canopy and biomass estimation,Animal or plant data
4,Not specified,Drones,Monitoring threats and forest inventory,Not specified,Exploring artificial intelligence for applicat...,Habitat degradation and biodiversity monitoring,Drone imagery
...,...,...,...,...,...,...,...
1677,Hong Kong and the Greater Bay Area,DNA barcode sequences,Biodiversity monitoring,"22.3193, 114.1694",A DNA barcode library for mangrove gastropods ...,Habitat degradation and biodiversity monitoring,Animal or plant data
1678,Amazon forest,Functional traits data,Understanding beta diversity dynamics,"-3.4653, -62.2159",Taxonomic and functional beta diversity of woo...,Habitat degradation and biodiversity monitoring,Animal or plant data
1679,South America,Occurrence records,Assessing tree species richness,"-15.7801, -47.9292",Dissecting a biodiversity hotspot: The importa...,Tree species classification,Animal or plant data
1680,Amazon forest,LiDAR remote sensing,Understanding seasonal dynamics of leaf area i...,"-3.4653, -62.2159",Seasonal and drought-related changes in leaf a...,Canopy and biomass estimation,Satellite imagery or LiDAR


In [29]:
len(output_df_to_save)

1682